In [5]:
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
import time

In [35]:
#initiate Selenium and accept Terms of Use
driver = webdriver.Chrome()

try:
    driver.get("https://qcor.cms.gov/")
    ActionChains(driver)\
        .send_keys(Keys.RETURN)\
        .perform()
except:
        pass

checkbox = driver.find_element(by=By.NAME, value="chkCertiCon")
checkbox.click()

button = driver.find_element(by=By.NAME, value="submit")
button.click()

#move through pages to the results of the RHC Active Provider query
time.sleep(2)

hospital_link = driver.find_element(by=By.CSS_SELECTOR, value='[href="report_select.jsp?which=3"]')
hospital_link.click()

time.sleep(2)

apsc_hospital = driver.find_element(by=By.CSS_SELECTOR, value='[href="hospital_wizard.jsp?which=3&report=active.jsp"]')
apsc_hospital.click()

time.sleep(2)

year_box = driver.find_element(by=By.NAME, value="year_type")
select_ybox = Select(year_box)
select_ybox.select_by_value("CY")

ptype_box = driver.find_element(by=By.NAME, value="view_cah")
select_ptype = Select(ptype_box)
select_ptype.deselect_by_visible_text("All Other Hospital Types")
select_ptype.select_by_visible_text("CAHs Only")

time.sleep(2)

run_report = driver.find_element(by=By.CSS_SELECTOR, value='[onclick="javascript: doSubmit();"]')
run_report.click()

time.sleep(2)
    
atl_link = driver.find_element(by=By.CSS_SELECTOR, value='[name="R04"]')
atl_link.click()

time.sleep(2)

tn_link = driver.find_element(by=By.CSS_SELECTOR, value='[name="STN"]')
tn_link.click()

time.sleep(2)

#create soup
content = driver.page_source
soup = BS(content)
driver.close()

#pull out the provider numbers
drill= soup.find_all("a", attrs={"class" : "drill"})
href = [x.get("href") for x in drill]
href_df = pd.DataFrame(href)
prov_nums = href_df[0].str.extract('(\d{6})')
prov_nums = prov_nums.rename(columns = {0 : 'prov_num'}).dropna()

In [36]:
#initiate Selenium and accept Terms of Use
driver = webdriver.Chrome()

try:
    driver.get("https://qcor.cms.gov/")
    ActionChains(driver)\
        .send_keys(Keys.RETURN)\
        .perform()
except:
        pass

checkbox = driver.find_element(by=By.NAME, value="chkCertiCon")
checkbox.click()

button = driver.find_element(by=By.NAME, value="submit")
button.click()

#Create lists for desired facility information
facility_name = []
cert_num = []
facility_type = []
address = []
city_state = []
phone = []
participation_date = []
accreditation_org = []
accreditation_type = []
number_beds = []
ownership_type = []
facility_subtype = []

for ind, row in prov_nums.iterrows():
    #access FQHC provider facility numbers
    prov_id = row['prov_num']
  
    #access page with facility details
    driver.get(f"https://qcor.cms.gov/active_popup.jsp?prvdr_intrnl_num={prov_id}&year_value=2022&jump=1&provider=01&which=3")
    
    #create soup
    content = driver.page_source
    soup = BS(content)
    
    #pull out the table that holds facility details
    tr = soup.find_all("tr")
    name = tr[1].find("td").text.strip()
    cnum = tr[2].find("td").text.strip()
    ftype = tr[3].find("td").text.strip()
    addr = tr[4].find("td").text.strip()
    cit_st = tr[5].text.strip()
    pho = tr[6].find("td").text.strip()
    pdate = tr[7].find("td").text.strip()
    aorg = tr[9].find("td").text.strip()
    atype = tr[10].find("td").text.strip()
    beds = tr[11].find("td").text.strip()
    otype = tr[12].find("td").text.strip()
    fsubtype = tr[14].find("td").text.strip()

    #append lists
    facility_name.append(name)
    cert_num.append(cnum)
    facility_type.append(ftype)
    address.append(addr)
    city_state.append(cit_st)
    phone.append(pho)
    participation_date.append(pdate)
    accreditation_org.append(aorg)
    accreditation_type.append(atype)
    number_beds.append(beds)
    ownership_type.append(otype)
    facility_subtype.append(fsubtype)
    
driver.close()

#create dictionary
cah_dict = {
    "facility_name" : facility_name,
    "cert_num" : cert_num,
    "facility_type" : facility_type,
    "facility_subtype" : facility_subtype,
    "address" : address,
    "city_state" : city_state,
    "phone" : phone,
    "participation_date" : participation_date,
    "ownership_type" : ownership_type,
    "accreditation_organization" : accreditation_org,
    "accreditation_type" : accreditation_type,
    "number_of_beds" : number_beds
}

#turn dictionary into a dataframe
cah_qcor_df = pd.DataFrame(cah_dict)

AttributeError: 'NoneType' object has no attribute 'text'

In [31]:
href

["javascript: doSubmit('rgn', '01', 0,0,0,0,0);",
 "javascript: doSubmit('rgn', '02', 0,0,0,0,0);",
 "javascript: doSubmit('rgn', '03', 0,0,0,0,0);",
 "javascript: doSubmit('rgn', '04', 0,0,0,0,0);",
 "javascript: doSubmit('state', '04', 'AL',0,0,0,0);",
 "javascript: doSubmit('state', '04', 'FL',0,0,0,0);",
 "javascript: doSubmit('state', '04', 'GA',0,0,0,0);",
 "javascript: doSubmit('state', '04', 'KY',0,0,0,0);",
 "javascript: doSubmit('state', '04', 'MS',0,0,0,0);",
 "javascript: doSubmit('state', '04', 'NC',0,0,0,0);",
 "javascript: doSubmit('state', '04', 'SC',0,0,0,0);",
 "javascript: doSubmit('state', '04', 'TN',0,0,0,0);",
 "javascript: doSubmit('provider', '04', 0, 0,2022,0,'444030');",
 "javascript: doSubmit('provider', '04', 0, 0,2022,0,'443038');",
 "javascript: doSubmit('provider', '04', 0, 0,2022,0,'441303');",
 "javascript: doSubmit('provider', '04', 0, 0,2022,0,'440048');",
 "javascript: doSubmit('provider', '04', 0, 0,2022,0,'440016');",
 "javascript: doSubmit('provid